# Import thư viện

In [1]:
import numpy as np
import pandas as pd
import ast
import re
import random
from tqdm.auto import tqdm

c:\Users\ADMIN\Desktop\Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Làm sạch từng nguồn và hợp nhất

- Loại bỏ các dòng null và duplicate
- Chỉnh sửa cho đúng định dạng
- Biến đổi thành template: (history, question, answer, input, output)

Ví dụ:

Người dùng: Bác sĩ ơi tôi có những nốt đỏ trên da. Đây là dấu hiệu gì vậy?

Bác sĩ AI: Đây là có lẽ là dấu hiệu…


In [2]:
def randomize_greeting(text, probability=0.7):
    greetings = [
        "Xin chào",
        "Hello",
        "Chào mừng",
        "Rất vui được gặp bạn",
        "Rất cảm ơn sự tin tưởng của bạn",
        "Chào bạn",
        "Chào em",
        "Xin chào bạn",
        "Xin chào em",
        "Chào bạn thân mến",
        "Rất vui được gặp bạn hôm nay",
        "Cảm ơn bạn đã liên hệ",
        "Chào bạn, tôi ở đây để giúp bạn",
        "Rất hân hạnh được hỗ trợ bạn"
    ]
    
    # Kiểm tra lời chào đầu câu và thay ngẫu nhiên với xác suất
    if text.startswith(("Chào em", "Chào bạn")) and random.random() < probability:
        greeting = random.choice(greetings)
        text = re.sub(r"^(Chào em|Chào bạn)", greeting, text, count=1)
    
    return text

def str2lst_of_dict(text):
    try:
        # Trực tiếp dùng chuỗi gốc mà không thay thế \n
        data = ast.literal_eval(text)
        return data
    except (ValueError, SyntaxError) as e:
        # In ra chi tiết lỗi để dễ dàng phát hiện vấn đề
        # print(f"Phát hiện lỗi!: {e} - Nội dung không hợp lệ: {text}")
        return None

def fix_format(conversation):
    if str2lst_of_dict(conversation.replace('\n', '\\n')) is not None:
        return conversation.replace('\n', '\\n')
    
    if not conversation.startswith('['):
        conversation = '[' + conversation
    if not conversation.endswith(']'):
        conversation = conversation + ']'
    conversation = conversation.replace('}', '},').replace(',,',',').replace('},]','}]')
    conversation = conversation.replace("},\n{", "}, {")
    conversation = conversation.replace("}, \n{", "}, {")
    conversation = conversation.replace("}]\n{", "}, {")
    conversation = conversation.replace('"\n}', '"}')
    conversation = conversation.replace('},\n]','}]').replace('"]','}]')
    conversation = conversation.replace('\n "role"','"role"')
    conversation = conversation.replace('\n "message"', '"message"')
    conversation = conversation.replace('\n', '\\n')
    conversation = conversation.replace("'",'"')
    conversation = conversation.replace('`','"')
    conversation = re.sub(r'("\s*:\s*"[^"]+")\s*(")', r'\1,\2', conversation)
    conversation = re.sub(r'\]+$', ']', conversation)
    
    if '"Bác sĩ":' in conversation:
        conversation = conversation.replace('"Bác sĩ":', '"Bác sĩ", "message":')
    if '"Người dùng":' in conversation:
        conversation = conversation.replace('"Người dùng":', '"Người dùng", "message":')
        
    # Trường hợp riêng
    conversation = conversation.replace('"mục"','mục').replace('"dv"','dv').replace('Paula"s',"Paulas").replace('u}]','"}]').replace('ẹ}]','"}]').replace('"cậu nhỏ"',"cậu nhỏ")
        
    return conversation
    
    

In [4]:
csv_path = ["data_augmentation.csv", "data_augmentation2.csv", "data_crawling.csv"]
data_dict = {
    "Question": [],
    "Answer": [],
}
total = 0

for path in csv_path:
    df = pd.read_csv(path)
    total += df.shape[0]
    df = df.dropna()  # Xóa các giá trị thiếu
    df = df.drop_duplicates()  # Loại bỏ các hàng trùng lặp

    # Xử lý từng hội thoại trong cột 'conversation'
    for conversation in tqdm(df["conversation"], desc=f"Processing {path}"):
        fix_conversation = fix_format(str(conversation))
        a = str2lst_of_dict(fix_conversation)  # Chuyển chuỗi thành danh sách dictionary của tin nhắn
        
        if a is None:
            continue

        try:
            question = ""

            for i, message in enumerate(a):
                role = message["role"]
                msg = message["message"].strip()

                if role == "Người dùng":
                    question += msg

                elif role == "Bác sĩ":
                    if question != "":
                        data_dict["Question"].append(question.strip())
                        data_dict["Answer"].append(msg.strip())
                        question = ""
        except Exception as e:
            print(f"Error processing conversation: {e}")
            continue
            
            

df = pd.DataFrame(data_dict)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
print("Hợp nhất thành công! Số điểm dữ liệu so với ban đầu:",str(len(df)),"/",str(total))
df.head(10)

Processing data_augmentation.csv:  23%|██▎       | 4751/20918 [00:00<00:01, 11040.38it/s]

Error processing conversation: 'message'
Error processing conversation: 'message'


Processing data_augmentation.csv:  33%|███▎      | 6834/20918 [00:00<00:01, 8654.04it/s] 

Error processing conversation: 'message'


Processing data_augmentation.csv:  76%|███████▌  | 15836/20918 [00:01<00:00, 9281.86it/s]

...


Processing data_augmentation2.csv:  59%|█████▉    | 17905/30182 [00:02<00:01, 7763.23it/s]

Error processing conversation: 'message'
Error processing conversation: 'ellipsis' object is not subscriptable


Processing data_crawling.csv: 100%|██████████| 1056/1056 [00:00<00:00, 8734.15it/s]


Hợp nhất thành công! Số điểm dữ liệu so với ban đầu: 65816 / 52659


,Question,Answer
0,"Chào BS,\nEm bị bệnh nấm da lâu năm không khỏi...","Chào bạn,\nNấm da là một loại bệnh dễ tái phát..."
1,"BS ơi, em bị nấm da ở vùng kín, em rất ngại đi...","Bạn thân mến, nấm da ở vùng kín có thể do nhiề..."
2,"Con gái em bị nấm da đầu, em đã mua thuốc bôi ...","Chào bạn, nấm da đầu ở trẻ em có thể do nhiều ..."
3,"Em bị nấm da, bác sĩ ơi, em có thể tự điều trị...","Chào bạn, việc tự điều trị nấm da có thể dẫn đ..."
4,"Mông của em bị thâm, có mụn mọc nhiều như rôm,...","Thân mến,\nTheo sinh lý thì mông của nhiều ngư..."
5,"Mông em bị thâm, có mụn mọc nhiều, không màu k...","Chào bạn, tình trạng da mông thâm, có mụn, khô..."
6,"Mông em bị thâm, có mụn mọc nhiều, không màu k...","Chào bạn, tình trạng da mông bị thâm, có mụn, ..."
7,"Em bị thâm mông, nhiều mụn nhỏ li ti màu trắng...","Chào bạn, tình trạng thâm mông, nhiều mụn nhỏ ..."
8,"Mông em bị thâm, có mụn mọc nhiều, không màu k...","Chào bạn, tình trạng thâm mông, mụn, dễ bị kíc..."
9,"Chào BS,\nMặt tôi hiện tại đang bị mụn đỏ và m...","Chào bạn,\nLăn kim là một phương pháp làm tái ..."


In [5]:
df.shape

(65816, 2)

# Save to CSV

In [6]:
df.to_csv("./dataset/augmentation_dataset.csv", index=False)